In [ ]:
import pandas as pd
from pyprojroot import here

import importlib
from conversion import *

In [ ]:
data = pd.read_csv(here() / 'data/raw/data_labels.csv', encoding='utf-8')
#columns = pd.read_csv(here() / 'data/raw/data.csv', encoding='utf-8').columns
data.shape[0]

In [ ]:
# Filter to complete
data = data[data['Complete?']=='Complete']
data = data.iloc[6:]
data.shape[0]

In [ ]:
# Country preprocessing and filters
data.Country = data.Country.str.capitalize()
data.Country = data.Country.str.rstrip()

data.Country = data.Country.replace(country_conversion)
data = data[~data.Country.isna()]

data['Country abbreviation'] = data.Country.replace(abbreviations)

In [ ]:
# Fix city names
data['City'] = data['City'].str.capitalize()
data['City'] = data['City'].str.rstrip()

pd.Series(data.City.unique()).sort_values().to_excel(here() / 'output/tables/cities.xlsx')

data['City'] = data['City'].replace(city_conversion)

In [ ]:
# Hospitals
data['Hospital Name'] = data['Hospital Name'].str.lstrip()
data['Hospital Name'] = data['Hospital Name'].str.rstrip()

data['Hospital Name'] = data['Hospital Name'].replace(hospital_conversion)

a = data[['City','Hospital Name']].sort_values(by=['City', 'Hospital Name'])
a.to_excel(here() / 'output/tables/hospitals.xlsx', index=False)

In [ ]:
a = data[['Last name', 'Country', 'City', 'Hospital Name', 'Emergency department name']].sort_values(by=['Country', 'City'])

a.to_excel(here() / 'output/tables/departments.xlsx', index=False)

In [ ]:
# Fix capitalization
data['First name'] = data['First name'].str.capitalize()
data['Middle name(s)'] = data['Middle name(s)'].str.capitalize()
data['Last name'] = data['Last name'].str.capitalize()

In [ ]:
# Modify based oan post hoc feedback
data.loc[data['Study ID']==302, 'phase_2']=1.0

In [ ]:
# Filter test entries
data = data[data['Study ID']!=26]

In [ ]:
data.iloc[:,80] = data.iloc[:,80].replace({'2-7days':'2-7 days'})

In [ ]:
# Remove duplicates
duplicates = (data['Last name'].duplicated() & data['First name'].duplicated() & data['Middle name(s)'].duplicated())
data = data[~duplicates]

In [ ]:
# Data types
categories = ['Unimportant', 'Slightly important', 'Moderately important', 'Important', 'Very important']
for col in data.iloc[:,30:60].columns:
    data[col] = pd.Categorical(data[col], categories=categories, ordered=True)

In [ ]:
data.to_csv(here() / 'data/base/data.csv', index=False)
data.to_pickle(here() / 'data/base/data.pkl')